# Capstone Project for the Data Science Specialization

### Rafael Y. Imai - April 2021

In [5]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

Firstly, let's load the datasets with the investment funds CNPJ (similar to a company ID in Brazil, can be used to identify uniquely a given investment fund),the share value at a given date, along with the volume of subscriptions and redemptions in Brazilian Reais:

In [12]:
historicSeries = pd.DataFrame()
for year in range(2017,2021):
    for month in range(1,12):
        file = "CVMData/inf_diario_fi_{0}{1}.csv".format(year,str(month).zfill(2))
        historicSeries = historicSeries.append(pd.read_csv(file, sep = ";"))
    print("{} inserted.".format(year))
 

2017 inserted.
2018 inserted.
2019 inserted.
2020 inserted.


Also, let's load a dataset with all of the investment funds that have existed since the early 90's in Brazil. This file was also obtained from CVM.

In [9]:
fundList = pd.read_excel("cad_fi.xlsx")

In [14]:
print(fundList.shape)
print(historicSeries.shape)

(60378, 39)
(14053831, 9)


In [16]:
fundList.head(1)

,TP_FUNDO,CNPJ_FUNDO,DENOM_SOCIAL,DT_REG,DT_CONST,CD_CVM,DT_CANCEL,SIT,DT_INI_SIT,DT_INI_ATIV,...,ADMIN,PF_PJ_GESTOR,CPF_CNPJ_GESTOR,GESTOR,CNPJ_AUDITOR,AUDITOR,CNPJ_CUSTODIANTE,CUSTODIANTE,CNPJ_CONTROLADOR,CONTROLADOR
0,FACFIF,00.000.684/0001-21,DEUTSCHE BANK FDO APLIC QUOTAS FDO INV FINANCE...,2003-04-30,1994-12-20,19.0,2000-08-01,CANCELADA,2000-08-01,NaT,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


On the scope of this study, only common Investment Funds ("FI") will be considered. Every form of Investment Fund in Brazil is regulated by CVM, so funds that have a specific purpose (e.g., FUNCINE funds, special investment funds whose objective is to support the movie industry) and certain funds that are closed-end funds (such as FII - Fundos de Investimento Imobiliário or Real Estate Investment Funds, similar to REITs) will be discarded.



Also, in order to provide a consistent first sample, only funds that happened to exist between December 31, 2016 and today or existed in January 1, 2017 and were closed in 2021 were considered.

In [30]:
firstFundFilterLayer = fundList[((fundList["TP_FUNDO"]=="FI")
                                 &(((fundList["DT_REG"]<"2016-12-31")&(fundList["DT_CANCEL"]>"2020-12-31"))
                                   |((fundList["DT_REG"]<"2016-12-31")&(fundList["SIT"]=="EM FUNCIONAMENTO NORMAL"))))]

In [33]:
firstFundFilterLayer.head(1)

,TP_FUNDO,CNPJ_FUNDO,DENOM_SOCIAL,DT_REG,DT_CONST,CD_CVM,DT_CANCEL,SIT,DT_INI_SIT,DT_INI_ATIV,...,ADMIN,PF_PJ_GESTOR,CPF_CNPJ_GESTOR,GESTOR,CNPJ_AUDITOR,AUDITOR,CNPJ_CUSTODIANTE,CUSTODIANTE,CNPJ_CONTROLADOR,CONTROLADOR
5290,FI,00.017.024/0001-53,FUNDO DE INVESTIMENTO RENDA FIXA EXPONENCIAL,2005-03-31,1994-06-22,43613.0,NaT,EM FUNCIONAMENTO NORMAL,2005-03-18,1994-06-22,...,BANCO BRADESCO S.A.,PJ,62.375.134/0001-44,BRAM - BRADESCO ASSET MANAGEMENT S.A. DISTRIBU...,57.755.217/0001-29,KPMG AUDITORES INDEPENDENTES,60.746.948/0001-12,BANCO BRADESCO S.A.,60.746.948/0001-12,BANCO BRADESCO S.A.


Some peculiarities of the Brazilian investment fund market must be observed. Among the funds present on the first selection, there might be exclusive funds (funds with a single shareholder or a select set of shareholders, that may apply in illiquid assets and not be subjected to frequent subscriptions and redemptions). As one of the main objects of this work is to assess whether macroeconomic variables have a significant impact on the volume of subscriptions and redemptions, it is interesting to filter out funds that are not subject to a regular volume of subscriptions and redemptions.

In this case, I will use the portfolio of IHFA, a market index provided by ANBIMA, an association whose members are companies involved with the financial sector in Brazil. IHFA stands for *Índice de Hedge Funds ANBIMA*, ANBIMA Hedge Funds Index in a literal translation. The portfolio selection criteria involve, among other rules, a minimum of ten shareholders and the exclusion of such exclusive funds.

More info can be found (in portuguese) at this link: https://www.anbima.com.br/pt_br/informar/precos-e-indices/indices/ihfa.htm.

The portfolio of the last quarter of 2020 will be used to assemble a list of funds, as it corresponds roughly to the end of the selected period. I had to post-process the portfolio file manually, as only a general report including some statistics about the index and the portfolio composition is published. The original file can be found here https://www.anbima.com.br/data/files/04/35/26/3E/E9E057106A070057882BA2A8/IHFA%20-%204T20.xlsx, but I will use a manually assembled Excel spreadsheet to store the CNPJ and the hedge fund names.

In [36]:
ihfaFirstQuarter20 = pd.read_excel("ANBIMAData/1Q2020-IHFA.xlsx")
ihfaFirstQuarter20.head()

,CNPJ,Nome do Fundo
0,27.944.378/0001-08,ABSOLUTE ESTRATÉGIA VERTEX FUNDO DE INVESTIMEN...
1,28.947.266/0001-65,ABSOLUTE VERTEX ADVISORY FUNDO DE INVESTIMENTO...
2,18.422.272/0001-45,ABSOLUTE VERTEX CSHG FUNDO DE INVESTIMENTO EM ...
3,21.470.989/0001-77,ABSOLUTE VERTEX FUNDO DE INVESTIMENTO EM COTAS...
4,23.565.803/0001-99,ABSOLUTE VERTEX II FUNDO DE INVESTIMENTO EM CO...
